Код расчитан на то, что шейпы называются как в gdb. Нужно менять region и gdb_path. Результаты в папке unified_risks

In [1]:
import arcpy
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np
from math import log10, floor, isfinite

Структура файла:
OBJECTID, region, MO, risk_hNormTot, risk_hExpertTot, risk_hNormLive, risk_hExpertLive, risk_hNormTot_num, risk_hExpertTot_num, risk_hNormLive_num, risk_hExpertLive_num, где вместо risk - каждый из рисков из словаря, _num - числовое значение категории

In [2]:
# ========= PATHS =========
region = 'Yakutia'
gdb_path = r"C:\Users\sdfsg\Downloads\Yakutia_RiskProfile_10-10-2025\Yakutia_RiskProfile_08-10-2025\Yakutia_RiskProfile\Default.gdb"
RISKS_FDS = "risk"
output_base = r"C:\Users\sdfsg\Documents\work\data\gispy\perm"
output_dir = Path(output_base) / "unified_risks"
output_dir.mkdir(parents=True, exist_ok=True)

# ========= MAPPING & CONFIG =========
hazard_mapping = {
    "hazCold": "Cold",
    "hazDrought": "Drought", 
    "hazWildfire": "Wildfire",
    "hazFlood": "Flood",
    "hazFreez5": "Freeze",
    "hazHail": "Hail",
    "hazHeat": "Heat",
    "hazRain": "Precipitation",
    "hazThunderstorm": "Thunder",
    "hazWind": "Wind",
    "hazPermafrost": "Permafrost",
}

# Точный словарь для перевода категорий
category_mapping = {
    "неопасно": 0,
    "умерено опасно": 1,
    "опасно": 2,
    "весьма опасно": 3,
    "очень опасно": 4,
    "не опасно": 0,
    "умеренно опасно": 1,
}

# Колонки для преобразования
risk_columns = ["hNormTot", "hExpertTot", "hNormLive", "hExpertLive"]
base_columns = ["region", "MO"]  # Убрали OBJECTID из базовых колонок

# ========= HELPERS =========
def extract_short_code(fc_name: str) -> str:
    """Извлекает код опасности из названия feature class."""
    m = re.search(r"(haz[A-Za-z0-9]+)", fc_name)
    if m:
        return m.group(1)
    for key in hazard_mapping.keys():
        if key.lower() in fc_name.lower():
            return key
    return fc_name

def convert_category_to_numeric(category):
    """Точно преобразует текстовую категорию в числовое значение."""
    if category is None:
        return None
    category_str = str(category).strip()
    for text_cat, num_cat in category_mapping.items():
        if text_cat == category_str:
            return num_cat
    return None

def get_feature_classes():
    """Собирает все feature classes с рисками."""
    hazard_data = {}
    
    arcpy.env.workspace = os.path.join(gdb_path, RISKS_FDS)
    print("Поиск feature classes в risk feature dataset...")
    
    for fc in arcpy.ListFeatureClasses() or []:
        name_l = fc.lower()
        if "template" in name_l or "perm_hazardtemplate_mo" in name_l:
            continue
        
        short = extract_short_code(fc)
        if short in hazard_mapping:  # Используем только риски из mapping, исключаем economics
            hazard_name = hazard_mapping[short]
            full_path = os.path.join(gdb_path, RISKS_FDS, fc)
            hazard_data[hazard_name] = full_path
            print(f"  ✓ Найден: {hazard_name} -> {fc}")
    
    print(f"\nВсего найдено рисков: {len(hazard_data)}")
    return hazard_data

def read_risk_data(fc_path, risk_name):
    """Читает данные риска и преобразует категории."""
    print(f"Чтение данных: {risk_name}")
    
    # Получаем все поля
    all_fields = [f.name for f in arcpy.ListFields(fc_path)]
    
    # Проверяем наличие базовых колонок (без OBJECTID)
    missing_base = [col for col in base_columns if col not in all_fields]
    if missing_base:
        print(f"  ⚠️ Отсутствуют базовые колонки: {missing_base}")
        return None
    
    # Определяем какие колонки риска есть в файле
    available_risk_cols = [col for col in risk_columns if col in all_fields]
    print(f"  Доступные колонки риска: {available_risk_cols}")
    
    # Собираем все нужные поля (без OBJECTID)
    fields_to_read = base_columns + available_risk_cols
    data = []
    
    with arcpy.da.SearchCursor(fc_path, fields_to_read) as cursor:
        for row in cursor:
            record = dict(zip(fields_to_read, row))
            
            # Добавляем название риска
            record['risk_name'] = risk_name
            
            # Преобразуем категориальные колонки в числовые
            for col in available_risk_cols:
                numeric_val = convert_category_to_numeric(record[col])
                if numeric_val is not None:
                    record[f"{col}_num"] = numeric_val
            
            data.append(record)
    
    df = pd.DataFrame(data)
    print(f"  Прочитано записей: {len(df)}")
    return df

def create_unified_dataframe(hazard_data):
    """Создает объединенную таблицу со всеми рисками."""
    all_data = []
    
    for risk_name, fc_path in hazard_data.items():
        df = read_risk_data(fc_path, risk_name)
        if df is not None and len(df) > 0:
            all_data.append(df)
    
    if not all_data:
        raise Exception("Не удалось прочитать данные ни по одному риску")
    
    # Объединяем все данные
    unified_df = pd.concat(all_data, ignore_index=True)
    print(f"\nОбъединенная таблица: {len(unified_df)} записей")
    
    return unified_df

def reshape_to_wide_format(df):
    """Преобразует данные в широкий формат (один MO = одна строка)."""
    print("Преобразование в широкий формат...")
    
    # Группируем по MO и создаем широкий формат
    wide_data = []
    
    # Сортируем муниципальные районы по алфавиту для последовательной нумерации
    sorted_mos = sorted(df['MO'].unique())
    
    for obj_id, mo in enumerate(sorted_mos, 1):  # Начинаем с 1
        mo_data = df[df['MO'] == mo]
        
        if len(mo_data) == 0:
            continue
            
        # Берем первую запись для базовых колонок
        base_record = mo_data.iloc[0][base_columns].to_dict()
        record = base_record.copy()
        
        # Добавляем новый OBJECTID
        record['OBJECTID'] = obj_id
        
        # Добавляем данные по каждому риску
        for risk in df['risk_name'].unique():
            risk_data = mo_data[mo_data['risk_name'] == risk]
            
            if len(risk_data) > 0:
                risk_row = risk_data.iloc[0]
                for col in risk_columns:
                    # Оригинальные текстовые значения
                    record[f"{risk}_{col}"] = risk_row.get(col)
                    # Числовые значения - преобразуем в целые числа
                    num_val = risk_row.get(f"{col}_num")
                    if num_val is not None and not pd.isna(num_val):
                        record[f"{risk}_{col}_num"] = int(num_val)
                    else:
                        record[f"{risk}_{col}_num"] = None
            else:
                # Если данных по этому риску нет, заполняем None
                for col in risk_columns:
                    record[f"{risk}_{col}"] = None
                    record[f"{risk}_{col}_num"] = None
        
        wide_data.append(record)
    
    wide_df = pd.DataFrame(wide_data)
    
    # Переупорядочиваем колонки, чтобы OBJECTID был первой
    cols = ['OBJECTID'] + base_columns + [col for col in wide_df.columns if col not in ['OBJECTID'] + base_columns]
    wide_df = wide_df[cols]
    
    # Преобразуем все числовые колонки в целый тип
    for col in wide_df.columns:
        if col.endswith('_num') and wide_df[col].dtype in ['float64', 'float32']:
            wide_df[col] = wide_df[col].astype('Int64')  # Int64 поддерживает NaN
    
    print(f"Широкий формат: {len(wide_df)} муниципальных районов")
    
    return wide_df

# ========= ОСНОВНОЙ КОД =========
def main():
    # Собираем feature classes
    hazard_data = get_feature_classes()
    
    if not hazard_data:
        print("Не найдено feature classes с рисками!")
        return
    
    # Создаем объединенную таблицу
    unified_df = create_unified_dataframe(hazard_data)
    
    # Преобразуем в широкий формат
    wide_df = reshape_to_wide_format(unified_df)
    
    # Сохраняем объединенную таблицу с припиской региона
    table_filename = f"{region}_unified_risks_data.csv"
    table_path = output_dir / table_filename
    wide_df.to_csv(table_path, index=False, encoding='utf-8-sig')
    print(f"✓ Объединенная таблица сохранена: {table_path}")
    
    # Дополнительно: сохраняем информацию о рисках
    risks_info = pd.DataFrame({
        'risk_name': list(hazard_data.keys()),
        'feature_class': [os.path.basename(path) for path in hazard_data.values()]
    })
    risks_info_path = output_dir / f"{region}_risks_info.csv"
    risks_info.to_csv(risks_info_path, index=False, encoding='utf-8-sig')
    print(f"✓ Информация о рисках сохранена: {risks_info_path}")
    
    # Выводим информацию о результате
    print(f"\n" + "="*60)
    print("ОБРАБОТКА ТАБЛИЦ ЗАВЕРШЕНА!")
    print(f"Выходная директория: {output_dir}")
    print(f"Муниципальных районов: {len(wide_df)}")
    print(f"Рисков: {len(hazard_data)}")
    
    print("\nПервые 10 районов с новыми ID:")
    for i, (_, row) in enumerate(wide_df.head(10).iterrows(), 1):
        print(f"  {row['OBJECTID']}: {row['MO']}")
    
    print("\nПример числовых значений:")
    num_cols = [col for col in wide_df.columns if col.endswith('_num')]
    for col in num_cols[:3]:  # Покажем первые 3 числовых колонки
        unique_vals = wide_df[col].dropna().unique()
        print(f"  {col}: {sorted(unique_vals)}")
    print("="*60)

if __name__ == "__main__":
    main()

Поиск feature classes в risk feature dataset...
  ✓ Найден: Heat -> Yakutia_hazHeat_MO
  ✓ Найден: Freeze -> Yakutia_hazFreez5_MO
  ✓ Найден: Drought -> Yakutia_hazDrought_MO
  ✓ Найден: Thunder -> Yakutia_hazThunderstorm_MO
  ✓ Найден: Hail -> Yakutia_hazHail_MO
  ✓ Найден: Precipitation -> Yakutia_hazRain_MO
  ✓ Найден: Wind -> Yakutia_hazWind_MO
  ✓ Найден: Cold -> Yakutia_hazCold_MO
  ✓ Найден: Flood -> Yakutia_hazFlood_MO
  ✓ Найден: Permafrost -> Yakutia_hazPermafrost_MO
  ✓ Найден: Wildfire -> Yakutia_hazWildfire_MO

Всего найдено рисков: 11
Чтение данных: Heat
  Доступные колонки риска: ['hNormTot', 'hExpertTot', 'hNormLive', 'hExpertLive']
  Прочитано записей: 36
Чтение данных: Freeze
  Доступные колонки риска: ['hNormTot', 'hExpertTot', 'hNormLive', 'hExpertLive']
  Прочитано записей: 36
Чтение данных: Drought
  Доступные колонки риска: ['hNormTot', 'hExpertTot', 'hNormLive', 'hExpertLive']
  Прочитано записей: 36
Чтение данных: Thunder
  Доступные колонки риска: ['hNormTot',